In [1]:
import sqlite3

DB_NAME = "google_repos.db"

conn = sqlite3.connect(DB_NAME)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS repositories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    language TEXT,
    stars INTEGER
)
""")

conn.commit()
conn.close()

print("DB created.")

DB created.


In [2]:
import sqlite3
import time
import requests
from bs4 import BeautifulSoup

DB_NAME = "google_repos.db"
BASE_URL = "https://github.com/google?tab=repositories"

headers = {
    "User-Agent": "Mozilla/5.0"
}

def get_repositories():
    repos = []
    url = BASE_URL

    while True:
        print(f"Fetching: {url}")
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")

        repo_items = soup.select("li.Box-row")   # ← GitHub の repo セレクタはこれ！

        for item in repo_items:

            # リポジトリ名
            name_tag = item.select_one("a[data-hovercard-type='repository']")
            name = name_tag.text.strip() if name_tag else "Unknown"

            # 言語
            lang_tag = item.select_one("span[itemprop='programmingLanguage']")
            language = lang_tag.text.strip() if lang_tag else "Unknown"

            # スター数（K を含む場合あり）
            star_tag = item.select_one("a[href$='/stargazers']")
            if star_tag:
                text = star_tag.text.strip()
                text = text.replace(",", "")
                if "k" in text.lower():
                    stars = int(float(text.lower().replace("k", "")) * 1000)
                else:
                    stars = int(text)
            else:
                stars = 0

            repos.append((name, language, stars))

            time.sleep(1)   # polite wait

        # 次ページがあるか？
        next_btn = soup.select_one("a.next_page")
        if next_btn and "disabled" not in next_btn.get("class", []):
            url = "https://github.com" + next_btn["href"]
        else:
            break

    return repos


def save_to_db(repos):
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()

    cur.executemany(
        "INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)",
        repos
    )
    conn.commit()
    conn.close()
    print("Saved to DB.")


if __name__ == "__main__":
    data = get_repositories()
    print("Fetched:", len(data))
    save_to_db(data)


Fetching: https://github.com/google?tab=repositories
Fetched: 10
Saved to DB.


In [3]:
import sqlite3

DB_NAME = "google_repos.db"

conn = sqlite3.connect(DB_NAME)
cur = conn.cursor()

cur.execute("SELECT name, language, stars FROM repositories ORDER BY stars DESC")

rows = cur.fetchall()

for r in rows:
    print(r)

conn.close()

('or-tools', 'C++', 12716)
('skia', 'C++', 10274)
('gemma.cpp', 'C++', 6623)
('perfetto', 'C++', 5022)
('XNNPACK', 'C', 2181)
('nomulus', 'Java', 1766)
('j2cl', 'Java', 1346)
('crubit', 'C++', 924)
('dawn', 'C++', 784)
('yggdrasil-decision-forests', 'C++', 622)
('heir', 'C++', 610)
('orbax', 'Python', 456)
('skia-buildbot', 'Go', 158)
('skia-buildbot', 'Go', 158)
('wasefire', 'Rust', 129)
('device-infra', 'Java', 58)
('dwh-migration-tools', 'Java', 54)
('koladata', 'C++', 27)
('dive', 'C++', 17)
('kotlin-fhirpath', 'Kotlin', 7)
